In [137]:
import os
import sys
import random
import json
import collections

import pandas as pd
import numpy as np
import scipy
import statsmodels

from tqdm import trange, tqdm_notebook as tqdm
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams["figure.figsize"] = (10,7)
pd.options.display.max_columns = 999

sns.set()

In [138]:
from annoy import AnnoyIndex

In [139]:
os.getcwd()

'/home/boris/study/UT study/ACM Sigmod/sigmod/notebooks'

In [140]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [141]:
from sigmod_src.utils import read_json, pprint_json, path_from_spec_id, get_vector_for_spec_id
from sigmod_src.data.make_dataset import make_specs_dataset, preprocess_specs_dataset, make_labelled_dataset
from sigmod_src.features.build_features import make_features
from sigmod_src.data.embedding import get_embedder, emb_lookup

# Load raw data

In [142]:
SPECS_PATH = '../data/raw/2013_camera_specs/'

In [143]:
LABELS_PATH = '../data/raw/sigmod_medium_labelled_dataset.csv'

# Make specs dataset

Extract fields from json, create one csv file

In [144]:
specs_df = make_specs_dataset(SPECS_PATH)

In [145]:
specs_df.shape

(29787, 2)

In [146]:
specs_df.head()

,spec_id,page_title
0,www.ebay.com//57656,Canon PowerShot ELPH 110 HS 16 1 MP Digital Ca...
1,www.ebay.com//60583,Canon Rebel 2000 35 mm Camera Great Condition ...
2,www.ebay.com//60440,Canon EOS Rebel T3i Digital SLR Camera 18 55mm...
3,www.ebay.com//24139,"GE C1033 10 1 MP Digital Camera 3X Zoom 2 4"" L..."
4,www.ebay.com//54903,Vivitar Clip Shot Digital Camera 1 1 MP | eBay


In [147]:
specs_df = preprocess_specs_dataset(specs_df)
specs_df.head()

Warning, dropped 4 rows containing null page titles


,spec_id,page_title,page_title_stem,brand,site
0,www.ebay.com//57656,Canon PowerShot ELPH 110 HS 16 1 MP Digital Ca...,canon powershot elph 110 hs 16 1 mp digit came...,canon,www.ebay.com
1,www.ebay.com//60583,Canon Rebel 2000 35 mm Camera Great Condition ...,canon rebel 2000 35 mm camera great condit wit...,canon,www.ebay.com
2,www.ebay.com//60440,Canon EOS Rebel T3i Digital SLR Camera 18 55mm...,canon eo rebel t3i digit slr camera 18 55mm is...,canon,www.ebay.com
3,www.ebay.com//24139,"GE C1033 10 1 MP Digital Camera 3X Zoom 2 4"" L...","ge c1033 10 1 mp digit camera 3x zoom 2 4"" lcd...",ge,www.ebay.com
4,www.ebay.com//54903,Vivitar Clip Shot Digital Camera 1 1 MP,vivitar clip shot digit camera 1 1 mp,vivitar,www.ebay.com


In [148]:
specs_df.head().page_title.values

array(['Canon PowerShot ELPH 110 HS 16 1 MP Digital Camera Black',
       'Canon Rebel 2000 35 mm Camera Great Condition with Case and Instruction Booklet',
       'Canon EOS Rebel T3i Digital SLR Camera 18 55mm Is 75 300mm III Lens Kit USA 609728170592',
       'GE C1033 10 1 MP Digital Camera 3X Zoom 2 4" LCD Black Brand New',
       'Vivitar Clip Shot Digital Camera 1 1 MP'], dtype=object)

In [149]:
specs_df.to_csv('../data/processed/specs_preprocessed.csv', index=None)

# Make embedding

In [150]:
emb, indexer = get_embedder(specs_df.page_title_stem, specs_df.spec_id)

/home/boris/anaconda3/lib/python3.6/site-packages/gensim/similarities/index.py:184: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  index = AnnoyIndex(num_features)


Quick test by eye

In [151]:
random_row = specs_df.sample(1).iloc[0]
text, spec_id = random_row.page_title_stem, random_row.spec_id

nns = emb_lookup(text, emb, indexer)

nn_pairs = []
for nn in nns:
    nn_spec_id, dist = nn
    nn_row = specs_df[specs_df.spec_id == nn_spec_id].iloc[0]
    nn_pairs.append((nn_row.spec_id, nn_row.page_title_stem))
print(f'[{spec_id}] ', text)
print('')
print('Most similar')
for pair in nn_pairs:
    print(f'[{pair[0]}] ', pair[1])

[www.price-hunt.com//9555]  olympus vg 150 point & shoot camera best price in india 2014, spec and review | valid in delhi, mumbai, bangalore, hyderabad, chennai, kolkata, ahmedabad, surat | price-hunt

Most similar
[www.price-hunt.com//9555]  olympus vg 150 point & shoot camera best price in india 2014, spec and review | valid in delhi, mumbai, bangalore, hyderabad, chennai, kolkata, ahmedabad, surat | price-hunt
[www.buzzillions.com//2642]  olympus vr-320 digit camera (silver) review | buzzillions.com
[www.ukdigitalcameras.co.uk//280]  olympus sh-1 digit camera (black) | uk digit camera
[www.ilgs.net//136]  olympus tg-850 - digit camera - photo - visual | ilgs.net
[www.price-hunt.com//858]  benq gh210 advanc point & shoot camera best price in india 2014, spec and review | valid in delhi, mumbai, bangalore, hyderabad, chennai, kolkata, ahmedabad, surat | price-hunt
[www.price-hunt.com//9856]  olympus vg 170 point & shoot camera best price in india 2014, spec and review | valid in delh

In [152]:
emb.save('../models/embedder.gensim')
indexer.save('../models/indexer.annoy')

# Make features for classifier

In [153]:
spec_features_df, vectorizers, site_le = make_features(specs_df)

In [154]:
spec_features_df.shape

(29783, 1002)

In [155]:
spec_features_df.head()

,page_title_stem__00,page_title_stem__013803117493,page_title_stem__0mp,page_title_stem__10,page_title_stem__10 30mm,page_title_stem__10 cctv,page_title_stem__10 mp,page_title_stem__100,page_title_stem__105mm,page_title_stem__1080p,page_title_stem__1080p hd,page_title_stem__1080p ip,page_title_stem__10mp,page_title_stem__10mp digit,page_title_stem__10x,page_title_stem__1100d,page_title_stem__12,page_title_stem__12 1mp,page_title_stem__12 megapixel,page_title_stem__12 mp,page_title_stem__12mp,page_title_stem__135,page_title_stem__135mm,page_title_stem__14,page_title_stem__14 42mm,page_title_stem__14 mp,page_title_stem__14mp,page_title_stem__15,page_title_stem__16,page_title_stem__16 1mp,page_title_stem__16 2mp,page_title_stem__16 50mm,page_title_stem__16 megapixel,page_title_stem__16 mp,page_title_stem__16gb,page_title_stem__16mp,page_title_stem__16mp digit,page_title_stem__16x,page_title_stem__18,page_title_stem__18 135mm,page_title_stem__18 55,page_title_stem__18 55mm,page_title_stem__18 mp,page_title_stem__18mp,page_title_stem__1d,page_title_stem__1mp,page_title_stem__1mp digit,page_title_stem__20,page_title_stem__20 mp,page_title_stem__200,page_title_stem__200mm,page_title_stem__2013,page_title_stem__2014,page_title_stem__2014 new,page_title_stem__2014 spec,page_title_stem__20x,page_title_stem__20x optic,page_title_stem__21,page_title_stem__24,page_title_stem__24 mp,page_title_stem__264,page_title_stem__28,page_title_stem__2cd2012,page_title_stem__2cd2032,page_title_stem__2cd2112,page_title_stem__2cd2112 buy,page_title_stem__2cd2132,page_title_stem__2cd2212,page_title_stem__2cd2212 i5,page_title_stem__2cd2232,page_title_stem__2cd2232 i5,page_title_stem__2cd2332,page_title_stem__2cd2532f,page_title_stem__2cd2632f,page_title_stem__2cd2632f is,page_title_stem__2cd3332,page_title_stem__2cd3332 hikvis,page_title_stem__2mp,page_title_stem__2mp digit,page_title_stem__30,page_title_stem__300,page_title_stem__300mm,page_title_stem__30m,page_title_stem__30mm,page_title_stem__30x,page_title_stem__32gb,page_title_stem__36,page_title_stem__3d,page_title_stem__3g,page_title_stem__3megapixel,page_title_stem__3mp,page_title_stem__3mp ds,page_title_stem__3mp ir,page_title_stem__3x,page_title_stem__3x optic,page_title_stem__40,page_title_stem__42,page_title_stem__42mm,page_title_stem__4x,page_title_stem__50,page_title_stem__50mm,page_title_stem__50mm len,page_title_stem__55,page_title_stem__55 mm,page_title_stem__55mm,page_title_stem__55mm is,page_title_stem__55mm len,page_title_stem__55mm vr,page_title_stem__5d,page_title_stem__5d mark,page_title_stem__5mp,page_title_stem__5x,page_title_stem__5x optic,page_title_stem__600d,page_title_stem__600d 18,page_title_stem__60d,page_title_stem__6d,page_title_stem__70,page_title_stem__700tvl,page_title_stem__70d,page_title_stem__720p,page_title_stem__720p hd,page_title_stem__720p ip,page_title_stem__75,page_title_stem__7d,page_title_stem__7d 18,page_title_stem__8gb,page_title_stem__8x,page_title_stem__90,page_title_stem__95,page_title_stem__99,page_title_stem__accessori,page_title_stem__action,page_title_stem__action camera,page_title_stem__af,page_title_stem__af dx,page_title_stem__ahd,page_title_stem__ahd camera,page_title_stem__ahmedabad,page_title_stem__ahmedabad surat,page_title_stem__alarm,page_title_stem__alibaba,page_title_stem__alibaba com,page_title_stem__alpha,page_title_stem__alpha nex,page_title_stem__alpha slt,page_title_stem__aluminum,page_title_stem__analog,page_title_stem__and,page_title_stem__and ixus,page_title_stem__and review,page_title_stem__android,page_title_stem__angl,page_title_stem__as,page_title_stem__at,page_title_stem__at buy,page_title_stem__at pc,page_title_stem__australia,page_title_stem__auto,page_title_stem__backpack,page_title_stem__bag,page_title_stem__bag buy,page_title_stem__bag camera,page_title_stem__bag case,page_title_stem__bag digit,page_title_stem__bag dslr,page_title_stem__bag for,page_title_stem__bag product,page_title_stem__bag waterproof,page_title_stem

In [156]:
spec_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29783 entries, 0 to 29786
Columns: 1002 entries, page_title_stem__00 to spec_id
dtypes: float64(1000), int64(1), object(1)
memory usage: 229.2+ MB


In [157]:
spec_features_df = specs_df.merge(spec_features_df, on='spec_id', how='left')

In [158]:
spec_features_df.head()

,spec_id,page_title,page_title_stem,brand,site,page_title_stem__00,page_title_stem__013803117493,page_title_stem__0mp,page_title_stem__10,page_title_stem__10 30mm,page_title_stem__10 cctv,page_title_stem__10 mp,page_title_stem__100,page_title_stem__105mm,page_title_stem__1080p,page_title_stem__1080p hd,page_title_stem__1080p ip,page_title_stem__10mp,page_title_stem__10mp digit,page_title_stem__10x,page_title_stem__1100d,page_title_stem__12,page_title_stem__12 1mp,page_title_stem__12 megapixel,page_title_stem__12 mp,page_title_stem__12mp,page_title_stem__135,page_title_stem__135mm,page_title_stem__14,page_title_stem__14 42mm,page_title_stem__14 mp,page_title_stem__14mp,page_title_stem__15,page_title_stem__16,page_title_stem__16 1mp,page_title_stem__16 2mp,page_title_stem__16 50mm,page_title_stem__16 megapixel,page_title_stem__16 mp,page_title_stem__16gb,page_title_stem__16mp,page_title_stem__16mp digit,page_title_stem__16x,page_title_stem__18,page_title_stem__18 135mm,page_title_stem__18 55,page_title_stem__18 55mm,page_title_stem__18 mp,page_title_stem__18mp,page_title_stem__1d,page_title_stem__1mp,page_title_stem__1mp digit,page_title_stem__20,page_title_stem__20 mp,page_title_stem__200,page_title_stem__200mm,page_title_stem__2013,page_title_stem__2014,page_title_stem__2014 new,page_title_stem__2014 spec,page_title_stem__20x,page_title_stem__20x optic,page_title_stem__21,page_title_stem__24,page_title_stem__24 mp,page_title_stem__264,page_title_stem__28,page_title_stem__2cd2012,page_title_stem__2cd2032,page_title_stem__2cd2112,page_title_stem__2cd2112 buy,page_title_stem__2cd2132,page_title_stem__2cd2212,page_title_stem__2cd2212 i5,page_title_stem__2cd2232,page_title_stem__2cd2232 i5,page_title_stem__2cd2332,page_title_stem__2cd2532f,page_title_stem__2cd2632f,page_title_stem__2cd2632f is,page_title_stem__2cd3332,page_title_stem__2cd3332 hikvis,page_title_stem__2mp,page_title_stem__2mp digit,page_title_stem__30,page_title_stem__300,page_title_stem__300mm,page_title_stem__30m,page_title_stem__30mm,page_title_stem__30x,page_title_stem__32gb,page_title_stem__36,page_title_stem__3d,page_title_stem__3g,page_title_stem__3megapixel,page_title_stem__3mp,page_title_stem__3mp ds,page_title_stem__3mp ir,page_title_stem__3x,page_title_stem__3x optic,page_title_stem__40,page_title_stem__42,page_title_stem__42mm,page_title_stem__4x,page_title_stem__50,page_title_stem__50mm,page_title_stem__50mm len,page_title_stem__55,page_title_stem__55 mm,page_title_stem__55mm,page_title_stem__55mm is,page_title_stem__55mm len,page_title_stem__55mm vr,page_title_stem__5d,page_title_stem__5d mark,page_title_stem__5mp,page_title_stem__5x,page_title_stem__5x optic,page_title_stem__600d,page_title_stem__600d 18,page_title_stem__60d,page_title_stem__6d,page_title_stem__70,page_title_stem__700tvl,page_title_stem__70d,page_title_stem__720p,page_title_stem__720p hd,page_title_stem__720p ip,page_title_stem__75,page_title_stem__7d,page_title_stem__7d 18,page_title_stem__8gb,page_title_stem__8x,page_title_stem__90,page_title_stem__95,page_title_stem__99,page_title_stem__accessori,page_title_stem__action,page_title_stem__action camera,page_title_stem__af,page_title_stem__af dx,page_title_stem__ahd,page_title_stem__ahd camera,page_title_stem__ahmedabad,page_title_stem__ahmedabad surat,page_title_stem__alarm,page_title_stem__alibaba,page_title_stem__alibaba com,page_title_stem__alpha,page_title_stem__alpha nex,page_title_stem__alpha slt,page_title_stem__aluminum,page_title_stem__analog,page_title_stem__and,page_title_stem__and ixus,page_title_stem__and review,page_title_stem__android,page_title_stem__angl,page_title_stem__as,page_title_stem__at,page_title_stem__at buy,page_title_stem__at pc,page_title_stem__australia,page_title_stem__auto,page_title_stem__backpack,page_title_stem__bag,page_title_stem__bag buy,page_title_stem__bag camera,page_title_stem__bag case,page_title_stem__bag digit,page_title_stem__bag dslr,page_title_stem__bag for,page_title_stem__bag product,p

In [159]:
spec_features_df.to_csv('../data/processed/specs_features.csv', index=None)

# Make labelled dataset

In [160]:
labels_df = make_labelled_dataset(LABELS_PATH, spec_features_df)
labels_df.shape

(46665, 2005)

In [161]:
labels_df.head()

,left_spec_id,right_spec_id,label,left_page_title_stem__00,left_page_title_stem__013803117493,left_page_title_stem__0mp,left_page_title_stem__10,left_page_title_stem__10 30mm,left_page_title_stem__10 cctv,left_page_title_stem__10 mp,left_page_title_stem__100,left_page_title_stem__105mm,left_page_title_stem__1080p,left_page_title_stem__1080p hd,left_page_title_stem__1080p ip,left_page_title_stem__10mp,left_page_title_stem__10mp digit,left_page_title_stem__10x,left_page_title_stem__1100d,left_page_title_stem__12,left_page_title_stem__12 1mp,left_page_title_stem__12 megapixel,left_page_title_stem__12 mp,left_page_title_stem__12mp,left_page_title_stem__135,left_page_title_stem__135mm,left_page_title_stem__14,left_page_title_stem__14 42mm,left_page_title_stem__14 mp,left_page_title_stem__14mp,left_page_title_stem__15,left_page_title_stem__16,left_page_title_stem__16 1mp,left_page_title_stem__16 2mp,left_page_title_stem__16 50mm,left_page_title_stem__16 megapixel,left_page_title_stem__16 mp,left_page_title_stem__16gb,left_page_title_stem__16mp,left_page_title_stem__16mp digit,left_page_title_stem__16x,left_page_title_stem__18,left_page_title_stem__18 135mm,left_page_title_stem__18 55,left_page_title_stem__18 55mm,left_page_title_stem__18 mp,left_page_title_stem__18mp,left_page_title_stem__1d,left_page_title_stem__1mp,left_page_title_stem__1mp digit,left_page_title_stem__20,left_page_title_stem__20 mp,left_page_title_stem__200,left_page_title_stem__200mm,left_page_title_stem__2013,left_page_title_stem__2014,left_page_title_stem__2014 new,left_page_title_stem__2014 spec,left_page_title_stem__20x,left_page_title_stem__20x optic,left_page_title_stem__21,left_page_title_stem__24,left_page_title_stem__24 mp,left_page_title_stem__264,left_page_title_stem__28,left_page_title_stem__2cd2012,left_page_title_stem__2cd2032,left_page_title_stem__2cd2112,left_page_title_stem__2cd2112 buy,left_page_title_stem__2cd2132,left_page_title_stem__2cd2212,left_page_title_stem__2cd2212 i5,left_page_title_stem__2cd2232,left_page_title_stem__2cd2232 i5,left_page_title_stem__2cd2332,left_page_title_stem__2cd2532f,left_page_title_stem__2cd2632f,left_page_title_stem__2cd2632f is,left_page_title_stem__2cd3332,left_page_title_stem__2cd3332 hikvis,left_page_title_stem__2mp,left_page_title_stem__2mp digit,left_page_title_stem__30,left_page_title_stem__300,left_page_title_stem__300mm,left_page_title_stem__30m,left_page_title_stem__30mm,left_page_title_stem__30x,left_page_title_stem__32gb,left_page_title_stem__36,left_page_title_stem__3d,left_page_title_stem__3g,left_page_title_stem__3megapixel,left_page_title_stem__3mp,left_page_title_stem__3mp ds,left_page_title_stem__3mp ir,left_page_title_stem__3x,left_page_title_stem__3x optic,left_page_title_stem__40,left_page_title_stem__42,left_page_title_stem__42mm,left_page_title_stem__4x,left_page_title_stem__50,left_page_title_stem__50mm,left_page_title_stem__50mm len,left_page_title_stem__55,left_page_title_stem__55 mm,left_page_title_stem__55mm,left_page_title_stem__55mm is,left_page_title_stem__55mm len,left_page_title_stem__55mm vr,left_page_title_stem__5d,left_page_title_stem__5d mark,left_page_title_stem__5mp,left_page_title_stem__5x,left_page_title_stem__5x optic,left_page_title_stem__600d,left_page_title_stem__600d 18,left_page_title_stem__60d,left_page_title_stem__6d,left_page_title_stem__70,left_page_title_stem__700tvl,left_page_title_stem__70d,left_page_title_stem__720p,left_page_title_stem__720p hd,left_page_title_stem__720p ip,left_page_title_stem__75,left_page_title_stem__7d,left_page_title_stem__7d 18,left_page_title_stem__8gb,left_page_title_stem__8x,left_page_title_stem__90,left_page_title_stem__95,left_page_title_stem__99,left_page_title_stem__accessori,left_page_title_stem__action,left_page_title_stem__action camera,left_page_title_stem__af,left_page_title_stem__af dx,left_page_title_stem__ahd,left_page_title_stem__ahd camera,left_page_title_stem__ahmedabad,left_page_title_stem__ahmedabad surat,left_page

In [162]:
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46665 entries, 0 to 46664
Columns: 2005 entries, left_spec_id to right_site_enc
dtypes: float64(2000), int64(3), object(2)
memory usage: 714.2+ MB


In [163]:
labels_df.to_csv('../data/processed/labelled_features.csv', index=None)